In [4]:
# numeric keypad to open a door
# Robot 1 uses a directional keypad to carry out the shortest number of keystrokes to open the door
# Robot 2 has another directional keypad to control Robot 1's movement on its own directional keypad
# Finally, you have your own directional keypad to control Robot 2's movement on it's own directional keypad

# shortest sequence on the first directional keypad == shortest sequence on your directional keypad?
# NO THIS IS NOT TRUE LOL WE NEED TO FIND ALL SHORTEST PATHS AND USE THEM

# complexity of each code: length of the shortest sequence of button presses on your numeric keypad * numeric part of code (ignoring leading zeros)
# get total sum of complexities of five codes on the list
from collections import deque
from itertools import product
import re 
class Part1:
    bfs_cache = {}
    # numeric and directional keypads are consistent throughout, store them as class variables
    numpad = [['7', '8', '9'], 
              ['4', '5', '6'], 
              ['1', '2', '3'], 
              ['', '0', 'A']]

    dirpad = [['', '^', 'A'], 
              ['<', 'v', '>']]

    def __init__(self, input_str: str):
        # input only contains codes, so that's all we'll store in the class
        self.codes = [code for code in input_str.split('\n')]

    def getKeyPosition(self, key: str, keypad: list):
        for row_idx, row in enumerate(keypad):
            for col_idx, col in enumerate(row):
                if col == key:
                    return row_idx, col_idx

    # derive a function to get neighboring keys on either type of pad
    # should also return the direction of the neighbor relative to the current key using the dirpad conventions
    def getNeighboringKeys(self, curr: dict, keypad: list):
        # curr dict will contain: the key (str), the coordinates of the key on the grid (tuple) and direction (str, empty string for curr)
        row_lim = len(keypad)-1
        col_lim = len(keypad[0])-1
        (row, col) = curr['pos']
        (u, d, l, r) = (((row-1, col), f"{curr['dir']}^"), ((row+1, col), f"{curr['dir']}v"), ((row, col-1), f"{curr['dir']}<"), ((row, col+1), f"{curr['dir']}>"))
        nbs = (u, d, l, r)
        valid_nbs = []
        for d in nbs:
            (r, c) = d[0]
            direction = d[-1]
            # make sure that neighboring key is not out of bounds and is a valid key (not empty)
            if 0 <= r <= row_lim and 0 <= c <= col_lim:
                key = keypad[r][c]
                if key:
                    valid_nbs.append({'key': key, 'pos': (r, c), 'dir': direction})

        return valid_nbs

    def BFS(self, start: str, end: str, keypad: list):
        if (start, end ) in self.bfs_cache:
            return self.bfs_cache[(start, end)]
        # Initialize a queue
        queue = deque()
        # Convert starting key into dictionary format
        start_pos = self.getKeyPosition(start, keypad)
        start_dict = {'key': start, 'pos': start_pos, 'dir': ""}
        # Add dict to queue
        queue.append(start_dict)

        # Initialize a list to store all shortest paths
        shortest_paths = []
        found_shortest = False  # Flag to indicate if the shortest paths have been found

        while queue:
            curr = queue.popleft()
            # Extract key and path
            key = curr['key']
            path = curr['dir']

            # If end is reached, add to shortest_paths and mark flag
            if key == end:
                shortest_paths.append(path)
                found_shortest = True

            # If the shortest paths are already found, stop processing further
            if found_shortest:
                continue

            # Get valid neighbors and add to the queue
            for nb in self.getNeighboringKeys(curr, keypad):
                queue.append(nb)

        self.bfs_cache[(start, end)] = shortest_paths
        return shortest_paths

    def getSequences(self, code: str, keypad: list):
        # Hand starts at A by default
        if not code.startswith('A'):
            code = f'A{code}'

        def getCodeBigrams(code: str):
            return [(code[idx], code[idx + 1]) for idx, _ in enumerate(code[:-1])]
        
        bigrams = getCodeBigrams(code)

        # Store all possible sequences as lists for combinations
        all_sequences = [[""]]

        for (start, end) in bigrams:
            # Get all shortest paths from BFS
            shortest_paths = self.BFS(start, end, keypad)  # Assumes BFS returns a list of paths
            # Append 'A' to each path (to reset to A)
            paths_with_reset = [f"{path}A" for path in shortest_paths]

            # Combine current paths with previous sequences
            all_sequences = [prev + [path] for prev in all_sequences for path in paths_with_reset]

        # Flatten sequences into strings
        shortest_seqs = [''.join(sequence) for sequence in all_sequences]
        # return only the shortest sequences
        return shortest_seqs
    
    # def getFinalSequence(self, code: str):
    #     first_seqs = self.getSequences(code, self.numpad)
    #     second_seqs = []
    #     for seq in first_seqs:
    #         second_seqs += self.getSequences(seq, self.dirpad)
    #     min_len = min([len(seq) for seq in second_seqs])
    #     second_seqs = [seq for seq in second_seqs if len(seq) == min_len]

    #     final_seqs = []
    #     for seq in second_seqs:
    #         final_seqs += self.getSequences(seq, self.dirpad)
    #     min_len = min([len(seq) for seq in final_seqs])
    #     final_seqs = [seq for seq in final_seqs if len(seq) == min_len]

    #     return final_seqs

    def getFinalSequence(self, code: str, num_iterations: int):
        # Start with the initial pad (numpad)
        current_seqs = self.getSequences(code, self.numpad)

        # Loop through the number of iterations, applying the next pad
        for i in range(num_iterations - 1):
            next_pad = self.dirpad  # After the first iteration, always use dirpad
            new_seqs = []
            for seq in current_seqs:
                new_seqs += self.getSequences(seq, next_pad)
            
            # Find the shortest sequences
            min_len = min([len(seq) for seq in new_seqs])
            current_seqs = [seq for seq in new_seqs if len(seq) == min_len]

        return current_seqs



    def getComplexity(self, code: str, seqs: list):
        min_len = min([len(seq) for seq in seqs])
        num_code = int(re.search('\d+', code).group())

        return min_len * num_code

    def getTotalComplexity(self, num_iterations: int):
        tc = 0
        for code in self.codes:
            seqs = self.getFinalSequence(code, num_iterations)
            tc += self.getComplexity(code, seqs)


        return tc

In [5]:
with open('data/test/21.txt', 'r', encoding='utf-8') as f:
    data = f.read()
part1 = Part1(data)
part1.getTotalComplexity(3)


126384

In [ ]:
with open('data/input/21.txt', 'r', encoding='utf-8') as f:
    data = f.read()
part1 = Part1(data)
part1.getTotalComplexity(3)


138764